In [1]:
# ADK's Built-in Toolbox: The Foundation
# Welcome 👋🏻 This notebook demonstrates how to use Google's Agent Development Kit (ADK) to build an agent
# that leverages a built-in tool, specifically `google_search`, to fetch live event data.
# This approach allows agents to provide dynamic and tailored responses to user queries.
# Example: A user asks, “What food festivals are happening in London next month?”
# The agent will use the `google_search` tool to find the latest events and then summarize them.

In [2]:
# load API Key set in .EnvironmentError
import dotenv
dotenv.load_dotenv()

import os
#import getpass

# Load the Gemini API key from environment variable
# If not set, prompt the user to enter it
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    print("GEMINI_API_KEY not found in environment variables.")
    #api_key = getpass.getpass("Enter your GEMINI_API_KEY: ")
    #os.environ["GEMINI_API_KEY"] = api_key

print("API key configured successfully." if api_key else "Failed to set API key.")

API key configured successfully.


In [3]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.18.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: c:\Users\gabri\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updat

In [5]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [6]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

# InMemorySessionService: A simple, in-memory service for managing conversation sessions.
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# types from google.genai: Provides data structures (like Content and Part) to represent
# messages exchanged with the language model.
from google.genai import types

# google_search: The built-in tool that allows the agent to perform web searches.
from google.adk.tools import google_search

# Markdown, display: Used for rendering rich text output in Jupyter notebooks.
from IPython.display import Markdown, display

In [7]:
# Modular Function for Agent Interaction
# This asynchronous function encapsulates the common logic for running an ADK agent interaction,
# making the main execution flow cleaner and more reusable.
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    # This flexibility allows for both isolated examples and continuous conversations.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. In a multi-turn conversation, you would reuse
    # the same session_id across turns to maintain context.
    # For these distinct examples, we'll use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object, which is how messages
    # are structured for the agent.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner: This is the engine that connects the user's message
    # to the agent and manages the session state.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent: This sends the user's message to the agent for processing.
    # The 'runner.run()' method returns an iterable of events that describe
    # the agent's actions and responses.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Process the events to find and extract the final response from the agent.
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            response = ''.join(part.text for part in event.content.parts if part.text)
            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

Modular function `run_adk_agent_interaction` successfully defined.


In [8]:
# Latest Events Agent: Definition
# The instruction set is the core of the agent, guiding its behavior.
# It should be specific, structured, and ensure safe operation.
latest_events_agent = Agent(
    name="latest_events_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that finds and summarizes current or upcoming events for a given destination and timeframe.",
    instruction="""
        You are a highly helpful and detailed travel assistant specialized in finding events.
        Your primary primary goal is to provide accurate, comprehensive, and well-organized information about events.

        When a user asks about events, follow these steps strictly:
        1.  **Identify Key Information:** Extract the desired event type (e.g., food festivals, music concerts, art exhibitions, sports events, cultural celebrations), destination, and timeframe (e.g., "next month", "August 2025", "next week", "soon").
        2.  **Use Google Search:** Utilize the `google_search` tool to find relevant current or upcoming events. Formulate precise search queries. For example, "food festivals London June 2025", "music concerts Berlin August 2025".
        3.  **Prioritize Official Sources:** Always aim to extract information from official event websites, reputable ticketing platforms, or well-known event listing sites.
        4.  **Summarize and Structure - IMPORTANT: FOLLOW THIS CLEANER FORMAT PRECISELY:**
            *   Start with a clear, friendly introduction.
            *   Present each event as a main item in a **Markdown unordered list (using asterisks `*`)**.
            *   Each event entry MUST follow this exact Markdown structure for clarity and spacing:
                ```
                *   **[Event Name]**
                    *   Description: [Brief Description].
                    *   Dates: [Start Date] - [End Date or Single Date].
                    *   Location: [Venue/Area], [City].
                    *   [More Info](URL) (Optional: if official URL found)
                ```
            *   Ensure only the "Event Name" is bolded as a main heading for each event.
            *   If dates are a range, use "Start Date - End Date". If single day, use "Single Date".
            *   Organize the events **chronologically by start date** if multiple events are found.

            **Example Event Formatting (pay close attention to line breaks and reduced bolding):**
            ```
            *   **Taste of London**
                *   Description: A renowned food festival featuring top restaurants and culinary experiences.
                *   Dates: June 18 - June 22, 2025.
                *   Location: Regent's Park, London.
                *   [More Info](https://www.tasteoflondon.co.uk/)

            *   **Berlin Lollapalooza**
                *   Description: A major music festival with diverse artists and stages.
                *   Dates: August 9 - August 10, 2025.
                *   Location: Olympiapark and Olympiastadion, Berlin.
                *   [More Info](https://www.lollapaloozade.com/)
            ```

        5.  **Handle Missing Information:**
            *   If the timeframe is vague (e.g., "soon", "what's happening?"), default to searching for events in the *upcoming 1-2 months* from the current date. If you find too many, suggest a more specific timeframe.
            *   If no specific event type is requested, search for a broad range of popular events (e.g., "major events [city] [month]").
        6.  **No Results Found:** If, after searching, no relevant events are found, politely state that no events matching the criteria could be found and offer to search for different dates or event types.
        7.  **Maintain Tone:** Be concise, friendly, and helpful.
""",
    tools=[google_search], # Assign the google_search tool to this agent.
)

print("`latest_events_agent` successfully defined with comprehensive instructions.")

`latest_events_agent` successfully defined with comprehensive instructions.


In [9]:
# Define common parameters for our agent interactions.
APP_NAME = "wanderwise_events_app"
USER_ID = "user_001"

# Create a common session service instance to reuse across multiple interactions.
common_session_service = InMemorySessionService()

# Define a dictionary of user inputs for various test cases.
# Each entry includes a descriptive name and the user's query.
EVENT_TEST_CASES = {
    "Normal Case: Food Festivals, London, Next Month (July 2025)":
        "What food festivals are happening in London next month?",

    "Specific Case: Music Concerts, Berlin, August 2025":
        "Music concerts in Berlin in August 2025.",

    "Vague Timeframe: General Events, Tokyo, Soon":
        "What's happening in Tokyo soon?",

    "Multiple Event Types: Cultural Events & Art Exhibitions, Paris, October 2025":
        "What are the major cultural events and art exhibitions happening in Paris in October 2025?",

    "Specific Event Type: Major Sports Events, New York City, July 2025":
        "Any major sports events in New York City in July 2025?",

    "No Results Expected: UFO Conventions, Hypothetical Location, Next Week (June 2025)":
        "Are there any major UFO conventions in Garetwasterland next week?",

    "Past Date Request: Festivals, Rio, January 2024":
        "What festivals happened in Rio in January 2024?", # Should clarify or state past

    "Future Date Request: Tech Conferences, San Francisco, December 2026":
        "Any big tech conferences in San Francisco in December 2026?",
}

print("------ [starting agent interactions] ------\n")

# Loop through each test case and run the agent.
for case_name, user_input in EVENT_TEST_CASES.items():
    # Create a unique session ID for each interaction to ensure isolated tests.
    session_id = f"{APP_NAME}_session_{case_name.replace(' ', '_').replace(',', '').lower()}"

    print(f"\n--- Testing >> {case_name} ---")
    print(f"User Input: '{user_input}'")

    # Invoke the latest_events_agent using the modular helper function.
    agent_response = await run_adk_agent_interaction(
        agent=latest_events_agent,
        user_id=USER_ID,
        session_id=session_id,
        input_text=user_input,
        app_name=APP_NAME,
        session_service=common_session_service # Pass the common session service
    )

    # Display the agent's response.
    display(Markdown(f"**Agent Response:**\n{agent_response}"))
    print(f"\n--- Testing Complete >> {case_name} ---")


print("\n------ [all agent interactions complete] ------")

------ [starting agent interactions] ------


--- Testing >> Normal Case: Food Festivals, London, Next Month (July 2025) ---
User Input: 'What food festivals are happening in London next month?'


**Agent Response:**
Here are a couple of food-related events happening in London next month:

*   **Hampton Court Palace Festive Fayre**
    *   Description: A Christmas shopping event with high-quality ingredients on offer to elevate winter feasts, street food stalls, pop-up bars, and festive drinks.
    *   Dates: December 5-7 & December 12-14, 2025.
    *   Location: Hampton Court Palace, London.
    *   [More Info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGNmlS9Fh4s3_OsWO2tl-xMhGmIzzbfEFeeg45ubkS-0OXM-_OocmBfwCsN0hfihp8vybUvH9Dl_nI70QTpX2XYeg-rGMH8LfkyEIC-UX_TmRKZuyGAB5JPdYEZB5RwtLsCGEtnequ1WGDpWTZo5YOCJBVOWsGZX6HfguSJweS7FXvq47Rm_gXnxRppD5mrhb_-hA==)

*   **Eat & Drink Festival, Ideal Home Show Christmas**
    *   Description: An indoor winter food festival with festive food and drink, live demos from top chefs, street food, and a Christmas market.
    *   Dates: November 26-30, 2025.
    *   Location: London.
    *   [More Info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExCs1i2h9fGPCVxRFG2M59pjEgH97mXgoLsblNxolVMIPyMS_fuvfxyG7Jj5zHDNVOJjrPEnL4GIZB8C12XdkmdSa8eYee3DBRba_rYDhIHNwOx4Wf-9HqhvBbFW4Chots-f1sfwQzsq1bLUr5ug_U7PX-s8M=)

*   **The London Taco Experience: Brixton Edition**
    *   Description: A two-day festival of Mexican food, drink, and music with taco traders, margaritas, and a hot sauce showcase.
    *   Dates: December (specific dates not provided in the search results).
    *   Location: Freight Brixton, London.




--- Testing Complete >> Normal Case: Food Festivals, London, Next Month (July 2025) ---

--- Testing >> Specific Case: Music Concerts, Berlin, August 2025 ---
User Input: 'Music concerts in Berlin in August 2025.'


**Agent Response:**
Here are some music events and festivals in Berlin in August 2025:

*   **Paula Hartmann • "kleine Feuer" Open Airs 2025**
    *   Description: Melancholic pop with sharp lyrics and a voice that gets under your skin. From sad girl melancholy to female rage.
    *   Dates: August 1, 2025.
    *   Location: Zitadelle Spandau, Berlin.

*   **Berlin Atonal**
    *   Description: A festival dedicated to sonic and visual arts, it brings together a variety of progressive and electronic musicians and visual artists. Unique, site-specific audio-visual displays are designed for the show and never repeated.
    *   Dates: August 27–31, 2025.
    *   Location: Kraftwerk complex, Berlin.

*   **Keinemusik celebrates live in Berlin**
    *   Description: Open-air party.
    *   Dates: August 23, 2025.
    *   Location: Tempelhof Airport, Berlin.

*   **Dance in August**
    *   Description: International contemporary dance festival that encourages experimental and innovative choreographies, with premiere performances as a part of the program.
    *   Dates: August 13–30, 2025.
    *   Location: Various locations in Berlin.

*   **Pop-Kultur Festival**
    *   Description: Live podcasts, performances and concerts with niche artists and newcomers.
    *   Dates: To be determined.
    *   Location: Berlin.

*   **Wall to Wall Club Festival 2025**
    *   Description: DJs from HEISSS and We Are Not Alone will fill the entire RSO - and there will also be rocking on the open-air floor.
    *   Dates: To be determined.
    *   Location: RSO, Berlin

*   **Sägewerk Festival 2025**
    *   Description: DJs (line-up tba) will provide booming beats to which you can dance yourself into a trance. There will be a live stage for the first time this year!
    *   Dates: To be determined.
    *   Location: Brandenburg, close to Berlin.



--- Testing Complete >> Specific Case: Music Concerts, Berlin, August 2025 ---

--- Testing >> Vague Timeframe: General Events, Tokyo, Soon ---
User Input: 'What's happening in Tokyo soon?'


...


--- Testing Complete >> No Results Expected: UFO Conventions, Hypothetical Location, Next Week (June 2025) ---

--- Testing >> Past Date Request: Festivals, Rio, January 2024 ---
User Input: 'What festivals happened in Rio in January 2024?'


**Agent Response:**
Here are some festivals and events that took place in Rio de Janeiro in January 2024:

*   **Carioca New Year's Festival**
    *   Description: Circuit Party.
    *   Dates: December 28, 2023 - January 2, 2024.
    *   Location: Rio de Janeiro.
    *   [More Info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEmWfGfI8aGzs1KQ6Hi2Ey7qPqhZyPHc_L6JxV5LVBkTCd9cIjk6_N-69phEeJWd0RaZhhO8dpLZ0K55yZB6hf5oUGmkgzJEDjzPYJY4VODW4emrIkljfGe5ond8gTcqKJxQ-xUUW3R4QXIaLoJ06KPILl-NnpL_Jy2OfV0iA==)

*   **Universo Spanta**
    *   Description: A music festival with a line-up of established artists and new talents, Carnival blocks and samba schools.
    *   Dates: January 6- January 20, 2024.
    *   Location: Marina da Glória, Rio de Janeiro.

*   **Breaking do Verão**
    *   Description: The largest breaking festival in the country, featuring national and international athletes.
    *   Dates: January 2024.
    *   Location: Lapa neighborhood, Rio de Janeiro.

*   **Juliana Silveira sings Floribella**
    *   Description: A tribute to the 20th anniversary of the soap opera Floribella, with a show, Q&A session and meet and greet.
    *   Dates: January 18, 2024.
    *   Location: Lagune Barra Hotel.

*   **Candlelight Concerts**
    *   Description: Candlelight shows in iconic places in Rio de Janeiro.
    *   Dates: January 16, 2024 - January 29, 2024.
    *   Location: Iconic places in Rio de Janeiro.




--- Testing Complete >> Past Date Request: Festivals, Rio, January 2024 ---

--- Testing >> Future Date Request: Tech Conferences, San Francisco, December 2026 ---
User Input: 'Any big tech conferences in San Francisco in December 2026?'


**Agent Response:**
Okay, I can help you with that. Here are a couple of tech conferences in San Francisco during December 2026.

*   **RSAC 2026 Conference**
    *   Description: Cybersecurity professionals unite to exchange ideas and tackle challenges.
    *   Dates: December 2025 - December 5, 2026 (Note: Registration open from Oct 20, 2025)
    *   Location: San Francisco, CA.
    *   [More Info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHAbNlUhLY9KnqntMrfkdEwLLhbeQgjwsRdv0auDfHE9t6gt1JrZsJ_8hG378haQNRqCTLnTIcFhfa1Q73J7vTZJ0qjfgOkfY3P0g49bRlZJ1h1VfLRRpZtinVzEw==)

*   **International Conference on Mental Health at the Workplace (ICMHW)**
    *   Description: Conference on Mental Health at the Workplace
    *   Dates: December 7, 2025
    *   Location: San Francisco, California, USA
    *   [More Info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEmcsOBpR0F6pwJ4-7roNdLpGNvcFz9ZahGzbF45TAoeGw-56WPkt7lVqawJXxliLATz5QeucQFC25aMM9ZX8svrD-aE9kiGGToGxKOThUN-JW6ZlvNHmdNWz0gqbj8OPX0DJMcdez2GHrwRgp8uSFt)

*   **International Conference on Advances in Migraine Sciences (ICAMS)**
    *   Description: Conference on Advances in Migraine Sciences.
    *   Dates: December 7, 2025
    *   Location: San Francisco, California, USA
    *   [More Info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEmcsOBpR0F6pwJ4-7roNdLpGNvcFz9ZahGzbF45TAoeGw-56WPkt7lVqawJXxliLATz5QeucQFC25aMM9ZX8svrD-aE9kiGGToGxKOThUN-JW6ZlvNHmdNWz0gqbj8OPX0DJMcdez2GHrwRgp8uSFt)




--- Testing Complete >> Future Date Request: Tech Conferences, San Francisco, December 2026 ---

------ [all agent interactions complete] ------


In [10]:
# Congratulations 🎉 
# You’ve just built and run an agent that uses the ADK built-in `google_search` tool
# to fetch live event data for a travel itinerary.
# This demonstrates the power of combining LLM reasoning with external tools to create
# truly intelligent and practical applications.
# Try modifying the prompt or agent instructions, and see how the results change!